In [2]:
# importation des biblio
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import os
from os import listdir

from sklearn.model_selection import train_test_split
import cv2
from keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from datetime import datetime
from PIL import Image
Image.MAX_IMAGE_PIXELS = None   # disables the warning

In [3]:
# chargement des donnees 
df=pd.read_csv('./exemple_essai/Flipkart/flipkart_com-ecommerce_sample_1050.csv')

In [4]:
df

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,55b85ea15a1536d46b7190ad6fff8ce7,2016-04-30 03:22:56 +0000,http://www.flipkart.com/elegance-polyester-mul...,Elegance Polyester Multicolor Abstract Eyelet ...,"[""Home Furnishing >> Curtains & Accessories >>...",CRNEG7BKMFFYHQ8Z,1899.0,899.0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,False,Key Features of Elegance Polyester Multicolor ...,No rating available,No rating available,Elegance,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
1,7b72c92c2f6c40268628ec5f14c6d590,2016-04-30 03:22:56 +0000,http://www.flipkart.com/sathiyas-cotton-bath-t...,Sathiyas Cotton Bath Towel,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEGFZHGBXPHZUH,600.0,449.0,7b72c92c2f6c40268628ec5f14c6d590.jpg,False,Specifications of Sathiyas Cotton Bath Towel (...,No rating available,No rating available,Sathiyas,"{""product_specification""=>[{""key""=>""Machine Wa..."
2,64d5d4a258243731dc7bbb1eef49ad74,2016-04-30 03:22:56 +0000,http://www.flipkart.com/eurospa-cotton-terry-f...,Eurospa Cotton Terry Face Towel Set,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEG6SHXTDB2A2Y,NaN,NaN,64d5d4a258243731dc7bbb1eef49ad74.jpg,False,Key Features of Eurospa Cotton Terry Face Towe...,No rating available,No rating available,Eurospa,"{""product_specification""=>[{""key""=>""Material"",..."
3,d4684dcdc759dd9cdf41504698d737d8,2016-06-20 08:49:52 +0000,http://www.flipkart.com/santosh-royal-fashion-...,SANTOSH ROYAL FASHION Cotton Printed King size...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJT9UQWHDUBH4,2699.0,1299.0,d4684dcdc759dd9cdf41504698d737d8.jpg,False,Key Features of SANTOSH ROYAL FASHION Cotton P...,No rating available,No rating available,SANTOSH ROYAL FASHION,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
4,6325b6870c54cd47be6ebfbffa620ec7,2016-06-20 08:49:52 +0000,http://www.flipkart.com/jaipur-print-cotton-fl...,Jaipur Print Cotton Floral King sized Double B...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJTHNGWVGWWQU,2599.0,698.0,6325b6870c54cd47be6ebfbffa620ec7.jpg,False,Key Features of Jaipur Print Cotton Floral Kin...,No rating available,No rating available,Jaipur Print,"{""product_specification""=>[{""key""=>""Machine Wa..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,958f54f4c46b53c8a0a9b8167d9140bc,2015-12-01 10:15:43 +0000,http://www.flipkart.com/oren-empower-extra-lar...,Oren Empower Extra Large Self Adhesive Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE88ZGTX65GH4V,1399.0,999.0,958f54f4c46b53c8a0a9b8167d9140bc.jpg,False,Oren Empower Extra Large Self Adhesive Sticker...,No rating available,No rating available,Oren Empower,"{""product_specification""=>[{""key""=>""Number of ..."
1046,fd6cbcc22efb6b761bd564c28928483c,2015-12-01 10:15:43 +0000,http://www.flipkart.com/wallmantra-large-vinyl...,Wallmantra Large Vinyl Sticker Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIEC889ZD5GDCVQ,4930.0,1896.0,fd6cbcc22efb6b761bd564c28928483c.jpg,False,Wallmantra Large Vinyl Sticker Sticker (Pack o...,No rating available,No rating available,Wallmantra,"{""product_specification""=>[{""key""=>""Number of ..."
1047,5912e037d12774bb73a2048f35a00009,2015-12-01 10:15:43 +0000,http://www.flipkart.com/uberlyfe-extra-large-p...,Uberlyfe Extra Large Pigmented Polyvinyl Films...,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE5UVGW2JWVWCT,4500.0,1449.0,5912e037d12774bb73a2048f35a00009.jpg,False,Buy Uberlyfe Extra Large Pigmented Polyvinyl F...,No rating available,No rating available,Uberlyfe,"{""product_specification""=>[{""key""=>""Number of ..."
1048,c3edc504d1b4f0ba6224fa53a43a7ad6,2015-12-01 10:15:43 +0000,http://www.flipkart.com/wallmantra-medium-viny...,Wallmantra Medium Vinyl Sticker Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >.

In [5]:
# la  variable categorie 
# Fonction pour extraire la catégorie
def extract_category(row):
    pos = str(row['product_category_tree']).find(">>")
    return str(row['product_category_tree'])[:pos]

# Appliquer la fonction à chaque ligne du DataFrame
df['category'] = df.apply(extract_category, axis=1)
df['category']=df['category'].str[2:-1]

In [6]:
df=df[['product_name', 'image', 'category']]

In [7]:
df['category'].unique()

array(['Home Furnishing', 'Baby Care', 'Watches',
       'Home Decor & Festive Needs', 'Kitchen & Dining',
       'Beauty and Personal Care', 'Computers'], dtype=object)

# Creation et entrainement du modele 

In [8]:
# Charger le modèle VGG16 pré-entraîné sans les couches fully connected à la fin
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Geler les couches du modèle pré-entraîné 
for layer in base_model.layers:
    layer.trainable = False

# le modèle
model = Sequential()

# Ajout le modèle VGG16 
model.add(base_model)

# Ajout des couches fully connected pour la classification
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax')) 

# Compilation
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# la structure du modèle
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 512)               12845568  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 7)                 3591      
                                                                 
Total params: 27563847 (105.15 MB)
Trainable params: 12849159 (49.02 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [31]:
# les données d'entraînement 
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    './train_images/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical', 
    shuffle=True)  

# Entraînement du modèle
start_time = datetime.now()
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10)
end_time = datetime.now()
training_time = end_time - start_time
print(f"Temps total d'entraînement : {training_time}")


Found 840 images belonging to 7 classes.
Epoch 1/10
26/26 [==============================] - 182s 7s/step - loss: 1.3671 - accuracy: 0.4889
Epoch 2/10
26/26 [==============================] - 187s 7s/step - loss: 0.6107 - accuracy: 0.8032
Epoch 3/10
26/26 [==============================] - 183s 7s/step - loss: 0.3895 - accuracy: 0.8713
Epoch 4/10
26/26 [==============================] - 185s 7s/step - loss: 0.2626 - accuracy: 0.9196
Epoch 5/10
26/26 [==============================] - 184s 7s/step - loss: 0.1767 - accuracy: 0.9592
Epoch 6/10
26/26 [==============================] - 182s 7s/step - loss: 0.1157 - accuracy: 0.9752
Epoch 7/10
26/26 [==============================] - 185s 7s/step - loss: 0.0949 - accuracy: 0.9814
Epoch 8/10
26/26 [==============================] - 182s 7s/step - loss: 0.0724 - accuracy: 0.9876
Epoch 9/10
26/26 [==============================] - 185s 7s/step - loss: 0.0525 - accuracy: 0.9950
Epoch 10/10
26/26 [==============================] - 182s 7s/step - 

#  evaluation du modele 

In [32]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    './test_images/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical', 
    shuffle=False)  

model.evaluate(
    test_generator,
    steps=test_generator.samples // test_generator.batch_size)


Found 210 images belonging to 7 classes.
2/6 [=========>....................] - ETA: 28s - loss: 0.9635 - accuracy: 0.7031

C:\Users\VORPC\datascientist\projet6\env\Lib\site-packages\PIL\Image.py:3182: DecompressionBombWarning: Image size (93680328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


6/6 [==============================] - 43s 7s/step - loss: 0.7215 - accuracy: 0.7708


[0.7215416431427002, 0.7708333134651184]

#  Classification avec data augmentation 

In [11]:
# Création d'un générateur d'images avec augmentation pour l'ensemble d'entraînement
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [11]:

train_generator = train_datagen.flow_from_directory(
    './train_images/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  
    shuffle=True)  

# Entraînement du modèle
start_time = datetime.now()
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=15)
end_time = datetime.now()
training_time = end_time - start_time
print(f"Temps total d'entraînement : {training_time}")


Found 840 images belonging to 7 classes.
Epoch 1/15


26/26 [==============================] - 301s 11s/step - loss: 1.6633 - accuracy: 0.3936
Epoch 2/15
26/26 [==============================] - 298s 11s/step - loss: 1.0776 - accuracy: 0.6077
Epoch 3/15
26/26 [==============================] - 302s 12s/step - loss: 0.9266 - accuracy: 0.6856
Epoch 4/15
26/26 [==============================] - 298s 11s/step - loss: 0.8448 - accuracy: 0.7017
Epoch 5/15
26/26 [==============================] - 298s 11s/step - loss: 0.7857 - accuracy: 0.7401
Epoch 6/15
26/26 [==============================] - 304s 12s/step - loss: 0.7298 - accuracy: 0.7364
Epoch 7/15
26/26 [==============================] - 299s 11s/step - loss: 0.7263 - accuracy: 0.7426
Epoch 8/15
26/26 [==============================] - 299s 11s/step - loss: 0.6676 - accuracy: 0.7723
Epoch 9/15
26/26 [==============================] - 319s 12s/step - loss: 0.6213 - accuracy: 0.7884
Epoch 10/15
26/26 [==============================] - 301s

In [12]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    './test_images/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical', 
    shuffle=False)   

model.evaluate(
    test_generator,
    steps=test_generator.samples // test_generator.batch_size)


Found 210 images belonging to 7 classes.
2/6 [=========>....................] - ETA: 45s - loss: 0.7646 - accuracy: 0.7656 

C:\Users\VORPC\datascientist\projet6\env\Lib\site-packages\PIL\Image.py:3182: DecompressionBombWarning: Image size (93680328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


6/6 [==============================] - 70s 11s/step - loss: 0.7433 - accuracy: 0.7604


[0.7432541847229004, 0.7604166865348816]

## modification nombre des epoques et batch_size

In [13]:

train_generator = train_datagen.flow_from_directory(
    './train_images/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical', 
    shuffle=True) 
# Entraînement du modèle

start_time = datetime.now()
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=15)

end_time = datetime.now()
training_time = end_time - start_time
print(f"Temps total d'entraînement : {training_time}")

Found 840 images belonging to 7 classes.
Epoch 1/15
13/13 [==============================] - 278s 21s/step - loss: 1.8529 - accuracy: 0.3286
Epoch 2/15
13/13 [==============================] - 265s 20s/step - loss: 1.2445 - accuracy: 0.5515
Epoch 3/15
13/13 [==============================] - 280s 21s/step - loss: 1.0194 - accuracy: 0.6469
Epoch 4/15
13/13 [==============================] - 292s 22s/step - loss: 0.8806 - accuracy: 0.6995
Epoch 5/15
13/13 [==============================] - 271s 20s/step - loss: 0.8352 - accuracy: 0.7320
Epoch 6/15
13/13 [==============================] - 281s 23s/step - loss: 0.8088 - accuracy: 0.7423
Epoch 7/15
13/13 [==============================] - 277s 21s/step - loss: 0.7901 - accuracy: 0.7345
Epoch 8/15
13/13 [==============================] - 271s 21s/step - loss: 0.7508 - accuracy: 0.7384
Epoch 9/15
13/13 [==============================] - 275s 21s/step - loss: 0.6607 - accuracy: 0.7784
Epoch 10/15
13/13 [==============================] - 307s 2

In [14]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    './test_images/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical', 
    shuffle=False)  



Found 210 images belonging to 7 classes.


In [21]:
from sklearn.metrics import confusion_matrix, classification_report

# prédictions sur les données de test
predictions = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size)

# Convertir les probabilités  en classes
predicted_classes = np.argmax(predictions, axis=-1)

# les vraies categories
true_classes = test_generator.classes[:len(predicted_classes)]  
# la matrice de confusion
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Matrice de confusion :")
print(conf_matrix)

# les métriques de classification 
class_report = classification_report(true_classes, predicted_classes, target_names=test_generator.class_indices.keys())
print("\nMétriques de classification par classe :")
print(class_report)


3/3 [==============================] - 45s 15s/step
Matrice de confusion :
[[19  1  4  1  3  1  1]
 [ 1 24  0  0  3  0  2]
 [ 0  3 26  1  0  0  0]
 [ 0  2  1 18  2  4  3]
 [ 5  0  0  1 24  0  0]
 [ 1  0  5  0  0 22  2]
 [ 0  0  0  0  0  0 12]]

Métriques de classification par classe :
                            precision    recall  f1-score   support

                 Baby Care       0.73      0.63      0.68        30
  Beauty and Personal Care       0.80      0.80      0.80        30
                 Computers       0.72      0.87      0.79        30
Home Decor & Festive Needs       0.86      0.60      0.71        30
           Home Furnishing       0.75      0.80      0.77        30
          Kitchen & Dining       0.81      0.73      0.77        30
                   Watches       0.60      1.00      0.75        12

                  accuracy                           0.76       192
                 macro avg       0.75      0.78      0.75       192
              weighted avg      